In [ ]:
#Import necessary libraries
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [ ]:
#Download every available years worth of play by play, weekly, injury, and season information

pbp_df = pd.DataFrame(nfl.import_pbp_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
schedules_df = pd.DataFrame(nfl.import_schedules([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))



In [ ]:
#Checking the columns in play by play
pbp_df.columns

In [ ]:
#Here, we aggregate each metric and group the result on the most relevant dimensions in the data set, although this list is not exhaustive and other dimensions may be added

passing_stats = pbp_df.groupby(['game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'passer_player_id', 'passer_player_name']).agg({
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum',
    'interception': 'sum',
    'was_pressure': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'fumble': 'sum'
}).reset_index()

#Checking the passing stats dataframe
passing_stats.head(15)

In [ ]:
#Writing the dataframe to csv
passing_stats.to_csv('nfl_passing_stats_1999_2024.csv')

In [ ]:
#and we can do the same for rushing, receiving:

rushing_stats = pbp_df.groupby(['game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'rusher_player_id', 'rusher_player_name']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'interception': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch': 'sum',
    'fumble': 'sum'
}).reset_index()

rushing_stats.head()


In [ ]:
rushing_stats.to_csv('nfl_rushing_stats_1999_2024.csv')

In [ ]:
receiver_stats = pbp_df.groupby(['game_id', 'game_date', 'week', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'receiver_player_id', 'receiver_player_name']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'interception': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch':'sum',
    'touchdown':'sum',
    'receiver_id': 'count',
    'fumble': 'sum'
}).reset_index()


receiver_stats.head()

In [ ]:
receiver_stats.to_csv('nfl_receiver_stats_1999_2024.csv')

In [ ]:
#kicking isn't as straightforward since the data doesn't have the same data points as are available with passing, rushing, and receiving:
kicker_plays = pbp_df[pbp_df['kicker_player_name'].notnull()]

# Filter out only successful field goals for longest field goal calculation
made_field_goals = kicker_plays[kicker_plays['field_goal_result'] == 'made']

# Aggregate kicker statistics including longest field goal and kickoff
kicker_stats = kicker_plays.groupby(['game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach','kicker_player_id', 'kicker_player_name']).agg({
    'field_goal_result': lambda x: (x == 'made').sum(),  # Sum of successful field goals
    'extra_point_result': lambda x: (x == 'good').sum(),  # Sum of successful extra points
    'kick_distance': 'sum',  # Sum of kickoff yardage
    'field_goal_attempt': 'sum',  # Total field goal attempts
    'kickoff_attempt': 'sum'
}).reset_index()

# Calculate longest successful field goal using the filtered made_field_goals DataFrame
longest_fg = made_field_goals.groupby(['game_id','kicker_player_id', 'kicker_player_name'])['kick_distance'].max().reset_index()
longest_fg.rename(columns={'kick_distance': 'longest_fg'}, inplace=True)
# Merge longest field goal into kicker_stats
kicker_stats = pd.merge(kicker_stats, longest_fg, on=['game_id', 'kicker_player_id', 'kicker_player_name'], how='left')

# Rename the aggregated columns for clarity
kicker_stats.rename(columns={
    'field_goal_attempt': 'fg_attempts',
    'field_goal_result': 'fg_made',
    'longest_fg': 'longest_fg',
    'extra_point_result': 'xp_made',
    'kick_distance': 'total_kick_distance'
}, inplace=True)

kicker_stats.drop(columns=['kicker_player_name'], inplace=True)
kicker_stats.head()

kicker_stats.to_csv('nfl_kicker_stats_1999_2024.csv')